<a href="https://colab.research.google.com/github/tesseract314/DS-Unit-4-Sprint-2-NLP/blob/master/LS_DS_423_Document_Classification_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2 Assignment 3*

# Document Classification

Use the following dataset of scraped "Data Scientist" and "Data Analyst" job listings to create your own Document Classification Models.

<https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-2-NLP/master/module3-Document-Classification/job_listings.csv>

Requirements:

- Apply both CountVectorizer and TfidfVectorizer methods to this data and compare results
- Use at least two different classification models to compare differences in model accuracy
- Try to "Hyperparameter Tune" your model by using different n_gram ranges, max_results, and data cleaning methods
- Try and get the highest accuracy possible!

In [38]:
from nltk.corpus import stopwords
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

pd.set_option('max_colwidth', 600)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [73]:
df = pd.read_csv('https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-2-NLP/master/module3-Document-Classification/job_listings.csv')
df.head()

,description,title,job
0,"b""<div><div>Job Requirements:</div><ul><li><p>\nConceptual understanding in Machine Learning models like Nai\xc2\xa8ve Bayes, K-Means, SVM, Apriori, Linear/ Logistic Regression, Neural, Random Forests, Decision Trees, K-NN along with hands-on experience in at least 2 of them</p>\n</li><li><p>Intermediate to expert level coding skills in Python/R. (Ability to write functions, clean and efficient data manipulation are mandatory for this role)</p>\n</li><li><p>Exposure to packages like NumPy, SciPy, Pandas, Matplotlib etc in Python or GGPlot2, dplyr, tidyR in R</p>\n</li><li><p>Ability to com...",Data scientist,Data Scientist
1,"b'<div>Job Description<br/>\n<br/>\n<p>As a Data Scientist 1, you will help us build machine learning models, data pipelines, and micro-services to help our clients navigate their healthcare journey. You will do so by empowering and improving the next generation of Accolade Applications and user experiences.</p><p><b>\nA day in the life\xe2\x80\xa6</b></p><ul><li>\nWork with a small agile team to design and develop mobile applications in an iterative fashion.</li><li>\nWork with a tight-knit group of development team members in Seattle.</li><li>\nContribute to best practices and help guide...",Data Scientist I,Data Scientist
2,"b'<div><p>As a Data Scientist you will be working on consulting side of our business. You will be responsible for analyzing large, complex datasets and identify meaningful patterns that lead to actionable recommendations. You will be performing thorough testing and validation of models, and support various aspects of the business with data analytics.</p><ul><li>\nAbility to do statistical modeling, build predictive models and leverage machine learning algorithms.</li><li>\nThis position will combine the typical Data Scientist math and analytical skills, with research, advanced business, co...",Data Scientist - Entry Level,Data Scientist
3,"b'<div class=""jobsearch-JobMetadataHeader icl-u-xs-mb--md""><div class=""jobsearch-JobMetadataHeader-item ""><span class=""icl-u-xs-mr--xs"">$4,969 - $6,756 a month</span></div><div class=""jobsearch-JobMetadataHeader-item icl-u-xs-mt--xs"">Contract</div></div><div>Under the general supervision of Professors Dana Mukamel and Kai Zheng, the incumbent will join the CalMHSA Mental Health Tech Suite Innovation (INN) Evaluation Team. This large, statewide multi-year study will evaluate the effectiveness of two new and innovative applications offered to people with mental health conditions, which inclu...",Data Scientist,Data Scientist
4,"b'<ul><li>Location: USA \xe2\x80\x93 multiple locations</li>\n<li>2+ years of Analytics experience</li>\n<li>Understand business requirements and technical requirements</li>\n<li>Can handle data extraction, preparation and transformation</li>\n<li>Create and implement data models</li></ul>'",Data Scientist,Data Scientist


In [59]:
df['description'][0]

'b"<div><div>Job Requirements:</div><ul><li><p>\\nConceptual understanding in Machine Learning models like Nai\\xc2\\xa8ve Bayes, K-Means, SVM, Apriori, Linear/ Logistic Regression, Neural, Random Forests, Decision Trees, K-NN along with hands-on experience in at least 2 of them</p>\\n</li><li><p>Intermediate to expert level coding skills in Python/R. (Ability to write functions, clean and efficient data manipulation are mandatory for this role)</p>\\n</li><li><p>Exposure to packages like NumPy, SciPy, Pandas, Matplotlib etc in Python or GGPlot2, dplyr, tidyR in R</p>\\n</li><li><p>Ability to communicate Model findings to both Technical and Non-Technical stake holders</p>\\n</li><li><p>Hands on experience in SQL/Hive or similar programming language</p>\\n</li><li><p>Must show past work via GitHub, Kaggle or any other published article</p>\\n</li><li><p>Master\'s degree in Statistics/Mathematics/Computer Science or any other quant specific field.</p></li></ul><div><div><div><div><div><d

In [0]:
df['description'] = df['description'].astype(str)
# df['description'] = df['description'].apply(word_tokenize)
# df.head()

In [11]:
df.dtypes

description    object
title          object
job            object
dtype: object

In [0]:
def clean_doc(doc):
    # split into tokens by white space
    
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens, and make lower
    tokens = [word.lower() for word in tokens if len(word) > 1]
    return tokens

In [0]:
df['description'] = df['description'].apply(clean_doc)

In [80]:
df.head()

,description,title,job
0,"[bdivdivjob, requirementsdivullipnconceptual, understanding, machine, learning, models, like, bayes, kmeans, svm, apriori, linear, logistic, regression, neural, random, forests, decision, trees, knn, along, handson, experience, least, thempnlilipintermediate, expert, level, coding, skills, pythonr, ability, write, functions, clean, efficient, data, manipulation, mandatory, rolepnlilipexposure, packages, like, numpy, scipy, pandas, matplotlib, etc, python, dplyr, tidyr, rpnlilipability, communicate, model, findings, technical, nontechnical, stake, holderspnliliphands, experience, sqlhive, s...",Data scientist,Data Scientist
1,"[bdivjob, descriptionbrnbrnpas, data, scientist, help, us, build, machine, learning, models, data, pipelines, microservices, help, clients, navigate, healthcare, journey, you, empowering, improving, next, generation, accolade, applications, user, experiencesppbna, day, small, agile, team, design, develop, mobile, applications, iterative, fashionlilinwork, tightknit, group, development, team, members, seattlelilincontribute, best, practices, help, guide, future, applicationslilinoperates, effectively, collaborative, member, development, teamlilinoperates, effectively, individual, quick, tur...",Data Scientist I,Data Scientist
2,"[bdivpas, data, scientist, working, consulting, side, business, you, responsible, analyzing, large, complex, datasets, identify, meaningful, patterns, lead, actionable, recommendations, you, performing, thorough, testing, validation, models, support, various, aspects, business, data, analyticspullinability, statistical, modeling, build, predictive, models, leverage, machine, learning, algorithmslilinthis, position, combine, typical, data, scientist, math, analytical, skills, research, advanced, business, communication, presentation, skillslilinprimary, job, location, sacramento, workfromho...",Data Scientist - Entry Level,Data Scientist
3,"[bdiv, classjobsearchjobmetadataheader, icluxsmbmddiv, classjobsearchjobmetadataheaderitem, span, monthspandivdiv, classjobsearchjobmetadataheaderitem, icluxsmtxscontractdivdivdivunder, general, supervision, professors, dana, mukamel, kai, zheng, incumbent, join, calmhsa, mental, health, tech, suite, innovation, inn, evaluation, team, this, large, statewide, multiyear, study, evaluate, effectiveness, two, new, innovative, applications, offered, people, mental, health, conditions, include, opportunities, online, chatting, users, online, listeners, responsibilities, incumbent, include, manag...",Data Scientist,Data Scientist
4,"[bullilocation, usa, multiple, years, analytics, experiencelinliunderstand, business, requirements, technical, requirementslinlican, handle, data, extraction, preparation, transformationlinlicreate, implement, data, modelsliul]",Data Scientist,Data Scientist


In [81]:
# join with space to make sklearn compatible
documents = []
for desc in df['description'].tolist():
  desc = " ".join(desc)
  documents.append(desc)
  
job = list(df['job'])
new_df = pd.DataFrame({'description': documents, 'job': job})
new_df.head()

,description,job
0,bdivdivjob requirementsdivullipnconceptual understanding machine learning models like bayes kmeans svm apriori linear logistic regression neural random forests decision trees knn along handson experience least thempnlilipintermediate expert level coding skills pythonr ability write functions clean efficient data manipulation mandatory rolepnlilipexposure packages like numpy scipy pandas matplotlib etc python dplyr tidyr rpnlilipability communicate model findings technical nontechnical stake holderspnliliphands experience sqlhive similar programming languagepnlilipmust show past work via gi...,Data Scientist
1,bdivjob descriptionbrnbrnpas data scientist help us build machine learning models data pipelines microservices help clients navigate healthcare journey you empowering improving next generation accolade applications user experiencesppbna day small agile team design develop mobile applications iterative fashionlilinwork tightknit group development team members seattlelilincontribute best practices help guide future applicationslilinoperates effectively collaborative member development teamlilinoperates effectively individual quick turnaround enhancements fixeslilinresponsible meeting expecta...,Data Scientist
2,bdivpas data scientist working consulting side business you responsible analyzing large complex datasets identify meaningful patterns lead actionable recommendations you performing thorough testing validation models support various aspects business data analyticspullinability statistical modeling build predictive models leverage machine learning algorithmslilinthis position combine typical data scientist math analytical skills research advanced business communication presentation skillslilinprimary job location sacramento workfromhome option ms phd relevant field computer science engineeri...,Data Scientist
3,bdiv classjobsearchjobmetadataheader icluxsmbmddiv classjobsearchjobmetadataheaderitem span monthspandivdiv classjobsearchjobmetadataheaderitem icluxsmtxscontractdivdivdivunder general supervision professors dana mukamel kai zheng incumbent join calmhsa mental health tech suite innovation inn evaluation team this large statewide multiyear study evaluate effectiveness two new innovative applications offered people mental health conditions include opportunities online chatting users online listeners responsibilities incumbent include managing analyzing text data created users two mental heal...,Data Scientist
4,bullilocation usa multiple years analytics experiencelinliunderstand business requirements technical requirementslinlican handle data extraction preparation transformationlinlicreate implement data modelsliul,Data Scientist


In [82]:
new_df.shape

(500, 2)

In [0]:
X = new_df['description']
y = new_df['job']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [90]:
vectorizer = CountVectorizer(max_features=None, ngram_range=(1,1), stop_words='english', 
                             min_df=0.1)

vectorizer.fit(X_train)

print(vectorizer.get_feature_names()[300:325])

['require', 'required', 'requirements', 'requires', 'research', 'responsibilities', 'responsible', 'results', 'right', 'role', 'sales', 'sas', 'scale', 'science', 'scientist', 'scientists', 'seeking', 'senior', 'service', 'services', 'sets', 'sex', 'sexual', 'similar', 'skills']


In [91]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(400, 393)


,ability,able,access,accuracy,achieve,action,actionable,ad,advanced,age,algorithms,amp,analyses,analysis,analyst,analysts,analytic,analytical,analytics,analyze,analyzing,andor,applicable,applicants,application,applications,applied,apply,applying,appropriate,areas,assist,attention,available,background,based,bdiv,benefits,best,better,bi,big,build,building,business,businesses,candidate,care,career,change,classjobsearchjobmetadataheader,classjobsearchjobmetadataheaderitem,client,clients,closely,cloud,code,collaborate,collaboration,collaborative,collection,color,committed,communicate,communication,companies,company,complete,complex,compliance,computer,concepts,consideration,core,create,creating,creative,critical,crossfunctional,culture,current,customer,customers,dashboards,data,database,databases,datadriven,datasets,day,decision,decisions,deep,define,degree,deliver,delivering,delivery,department,design,develop,developing,development,different,digital,directly,disability,diverse,diversity,drive,driven,duties,dynamic,economics,education,effectively,employees,employer,employment,engineering,engineers,ensure,enterprise,environment,equal,equivalent,excel,excellent,existing,experience,experienced,experiences,expertise,expression,external,federal,field,financial,findings,focus,focused,following,future,gender,global,goals,great,group,grow,growing,growth,hadoop,health,healthcare,help,helping,high,highly,icluxsmbmddiv,ideal,ideas,identify,identity,impact,implement,implementation,improve,improvement,improving,include,including,independently,individual,individuals,industry,information,initiatives,innovation,innovative,insights,insurance,integrity,intelligence,internal,issues,job,join,key,knowledge,language,languages,large,largest,lead,leadership,leading,learn,learning,level,life,like,local,looking,machine,maintain,make,making,manage,management,marital,market,marketing,math,mathematics,medical,meet,members,methods,metrics,microsoft,mining,mission,model,modeling,models,ms,multiple,national,need,needed,needs,new,offer,office,online,operational,operations,opportunities,opportunity,optimization,optimize,oral,organization,orientation,origin,partners,passion,passionate,people,perform,performance,physical,plan,platform,position,power,practices,predictive,preferred,present,presentation,problem,problems,process,processes,processing,product,products,professional,proficiency,program,programming,programs,project,projects,protected,provide,provides,providing,python,qualified,quality,quantitative,questions,quickly,race,range,real,receive,recommendations,record,regard,regression,related,relationships,relevant,religion,report,reporting,reports,require,required,requirements,requires,research,responsibilities,responsible,results,right,role,sales,sas,scale,science,scientist,scientists,seeking,senior,service,services,sets,sex,sexual,similar,skills,small,social,software,solutions,solve,solving,sources,spark,specific,sql,stakeholders,state,statistical,statistics,status,strategic,strategy,strong,subject,success,successful,support,supporting,systems,tableau,tasks,team,teams,technical,techniques,technologies,technology,test,testing,thinking,time,tools,track,training,trends,understand,understanding,unique,use,used,user,users,using,value,variety,various,verbal,veteran,vision,visualization,want,way,ways,web,work,working,workplace,world,writing,written,year,years,youll
0,0,1,1,0,0,0,0,0,1,0,0,0,2,1,0,1,3,1,2,0,2,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,17,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,3,0,0,1,0,0,0,2,3,1,0,0,0,0,0,0,0,0,0,0,2,0,1,1,0,1,1,2,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,2,0,2,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,

In [92]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(100, 393)


,ability,able,access,accuracy,achieve,action,actionable,ad,advanced,age,algorithms,amp,analyses,analysis,analyst,analysts,analytic,analytical,analytics,analyze,analyzing,andor,applicable,applicants,application,applications,applied,apply,applying,appropriate,areas,assist,attention,available,background,based,bdiv,benefits,best,better,bi,big,build,building,business,businesses,candidate,care,career,change,classjobsearchjobmetadataheader,classjobsearchjobmetadataheaderitem,client,clients,closely,cloud,code,collaborate,collaboration,collaborative,collection,color,committed,communicate,communication,companies,company,complete,complex,compliance,computer,concepts,consideration,core,create,creating,creative,critical,crossfunctional,culture,current,customer,customers,dashboards,data,database,databases,datadriven,datasets,day,decision,decisions,deep,define,degree,deliver,delivering,delivery,department,design,develop,developing,development,different,digital,directly,disability,diverse,diversity,drive,driven,duties,dynamic,economics,education,effectively,employees,employer,employment,engineering,engineers,ensure,enterprise,environment,equal,equivalent,excel,excellent,existing,experience,experienced,experiences,expertise,expression,external,federal,field,financial,findings,focus,focused,following,future,gender,global,goals,great,group,grow,growing,growth,hadoop,health,healthcare,help,helping,high,highly,icluxsmbmddiv,ideal,ideas,identify,identity,impact,implement,implementation,improve,improvement,improving,include,including,independently,individual,individuals,industry,information,initiatives,innovation,innovative,insights,insurance,integrity,intelligence,internal,issues,job,join,key,knowledge,language,languages,large,largest,lead,leadership,leading,learn,learning,level,life,like,local,looking,machine,maintain,make,making,manage,management,marital,market,marketing,math,mathematics,medical,meet,members,methods,metrics,microsoft,mining,mission,model,modeling,models,ms,multiple,national,need,needed,needs,new,offer,office,online,operational,operations,opportunities,opportunity,optimization,optimize,oral,organization,orientation,origin,partners,passion,passionate,people,perform,performance,physical,plan,platform,position,power,practices,predictive,preferred,present,presentation,problem,problems,process,processes,processing,product,products,professional,proficiency,program,programming,programs,project,projects,protected,provide,provides,providing,python,qualified,quality,quantitative,questions,quickly,race,range,real,receive,recommendations,record,regard,regression,related,relationships,relevant,religion,report,reporting,reports,require,required,requirements,requires,research,responsibilities,responsible,results,right,role,sales,sas,scale,science,scientist,scientists,seeking,senior,service,services,sets,sex,sexual,similar,skills,small,social,software,solutions,solve,solving,sources,spark,specific,sql,stakeholders,state,statistical,statistics,status,strategic,strategy,strong,subject,success,successful,support,supporting,systems,tableau,tasks,team,teams,technical,techniques,technologies,technology,test,testing,thinking,time,tools,track,training,trends,understand,understanding,unique,use,used,user,users,using,value,variety,various,verbal,veteran,vision,visualization,want,way,ways,web,work,working,workplace,world,writing,written,year,years,youll
0,1,0,0,2,0,1,0,0,1,1,0,1,0,0,0,2,0,2,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,0,1,0,1,0,2,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,2,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,9,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,4,0,0,0,2,0,1,0,2,0,1,0,0,0,0,0,0,0,3,0,0,0,1,2,0,0,0,0,0,1,0,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,3,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,2,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,2,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,2,0,1,0,0,0,0,0,1,0,0,0,0,0,2,1,0,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,2,0,2

#### Logistic Regression

In [95]:
lr = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = lr.predict(X_train_vectorized)
test_predictions = lr.predict(X_test_vectorized)

train_acc = accuracy_score(y_train, train_predictions)
test_acc = accuracy_score(y_test, test_predictions)

print('Train Accuracy:', train_acc)
print('Test Accuracy:', test_acc)

Train Accuracy: 0.9925
Test Accuracy: 0.9


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### Multinomial Naive Bayes

In [97]:
mnb = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = mnb.predict(X_train_vectorized)
test_predictions = mnb.predict(X_test_vectorized)

train_acc = accuracy_score(y_train, train_predictions)
test_acc = accuracy_score(y_test, test_predictions)

print('Train Accuracy:', train_acc)
print('Test Accuracy:', test_acc)

Train Accuracy: 0.9
Test Accuracy: 0.87


#### Random Forest Classifier

In [98]:
rfc = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = rfc.predict(X_train_vectorized)
test_predictions = rfc.predict(X_test_vectorized)

train_acc = accuracy_score(y_train, train_predictions)
test_acc = accuracy_score(y_test, test_predictions)

print('Train Accuracy:', train_acc)
print('Test Accuracy:', test_acc)

Train Accuracy: 0.9875
Test Accuracy: 0.91


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


# Stretch Goals

- Try some agglomerative clustering using cosine-similarity-distance! (works better with high dimensional spaces) robust clustering - Agglomerative clustering like Ward would be cool. Try and create an awesome Dendrogram of the most important terms from the dataset.

- Awesome resource for clustering stretch goals: 
 - Agglomerative Clustering with Scipy: <https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/>
 - Agglomerative Clustering for NLP: <http://brandonrose.org/clustering>
 
- Use Latent Dirichlet Allocation (LDA) to perform topic modeling on the dataset: 
 - Topic Modeling and LDA in Python: <https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24>
 - Topic Modeling and LDA using Gensim: <https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/>
